# Setting up gemini api key

In [ ]:
import google.generativeai as genai
genai.configure(api_key='YOUR_API_KEY')

# Available sicknesses

In [2]:
sicknesses = {
    "Common Cold": {
        "Description": "A viral infection of the upper respiratory tract, primarily affecting the nose and throat.",
        "Symptoms": [
            "Runny or stuffy nose",
            "Sneezing",
            "Sore throat",
            "Coughing",
            "Mild headache",
            "Fatigue",
            "Low-grade fever"
        ],
        "Causes": "Caused by various viruses, most commonly rhinoviruses. Spreads through respiratory droplets or contact with contaminated surfaces.",
        "Medications": [
            "Decongestants (e.g., pseudoephedrine) for nasal congestion",
            "Pain relievers (e.g., ibuprofen or acetaminophen) for fever and headaches",
            "Cough suppressants (e.g., dextromethorphan)",
            "Rest, hydration, and soothing throat lozenges"
        ]
    },
    "Influenza (Flu)": {
        "Description": "A highly contagious viral infection that affects the respiratory system.",
        "Symptoms": [
            "High fever",
            "Chills",
            "Muscle aches",
            "Fatigue",
            "Sore throat",
            "Dry cough",
            "Nasal congestion"
        ],
        "Causes": "Caused by influenza viruses transmitted via respiratory droplets or contaminated surfaces.",
        "Medications": [
            "Antiviral drugs (e.g., oseltamivir or zanamivir) if taken early",
            "Pain relievers (e.g., acetaminophen or ibuprofen)",
            "Rest, hydration, and over-the-counter medications for specific symptoms"
        ]
    },
    "Gastroenteritis (Stomach Flu)": {
        "Description": "An inflammation of the stomach and intestines causing digestive distress.",
        "Symptoms": [
            "Nausea",
            "Vomiting",
            "Diarrhea",
            "Abdominal cramps",
            "Fever",
            "Dehydration"
        ],
        "Causes": "Viral (e.g., norovirus, rotavirus), bacterial (e.g., E. coli, Salmonella), or parasitic infections; consuming contaminated food or water.",
        "Medications": [
            "Oral rehydration solutions (ORS) for dehydration",
            "Antidiarrheal drugs (e.g., loperamide, but not recommended for bacterial causes)",
            "Antiemetics (e.g., ondansetron) for nausea",
            "Probiotics to restore gut flora"
        ]
    },
    "Urinary Tract Infection (UTI)": {
        "Description": "An infection in any part of the urinary system, including the bladder, urethra, or kidneys.",
        "Symptoms": [
            "Pain or burning during urination",
            "Frequent urge to urinate",
            "Cloudy or foul-smelling urine",
            "Pelvic pain",
            "Sometimes fever"
        ],
        "Causes": "Bacterial infection, often Escherichia coli (E. coli), which enters through the urethra. Poor hygiene and dehydration increase risk.",
        "Medications": [
            "Antibiotics (e.g., nitrofurantoin, trimethoprim-sulfamethoxazole)",
            "Pain relievers (e.g., phenazopyridine) for urinary discomfort",
            "Increased water intake to flush out bacteria"
        ]
    },
    "Allergic Rhinitis (Hay Fever)": {
        "Description": "An allergic reaction to airborne allergens like pollen, dust mites, or pet dander.",
        "Symptoms": [
            "Sneezing",
            "Runny or congested nose",
            "Itchy eyes, throat, or nose",
            "Watery eyes",
            "Fatigue"
        ],
        "Causes": "Allergens trigger the immune system to release histamines, causing symptoms.",
        "Medications": [
            "Antihistamines (e.g., loratadine, cetirizine)",
            "Nasal corticosteroids (e.g., fluticasone, mometasone)",
            "Decongestants (e.g., pseudoephedrine) for temporary relief",
            "Allergen avoidance and air purifiers to reduce exposure"
        ]
    }
}


# Loading the retriever
(and testing it)

In [9]:
import faiss
import os 
import json
import numpy as np

def embed_query(text):
    embedding_result = genai.embed_content(
        model='models/embedding-001',
        content=text,
        task_type='retrieval_query'
    )
    return np.array(embedding_result['embedding'], dtype=np.float32).reshape(1, -1)

def retriever(query, index, sickness_names, k=1):
    query_embedding = embed_query(query)
    faiss.normalize_L2(query_embedding)
    distances, indices = index.search(query_embedding, k)
    retrieved_sickness = sickness_names[indices[0][0]]
    return retrieved_sickness

save_folder = "rag_system"
# Load index and mappings for retrieval
index = faiss.read_index(os.path.join(save_folder, "index.faiss"))
with open(os.path.join(save_folder, "documents.json"), "r") as f:
    docs_mapping = json.load(f)
sickness_names = [docs_mapping[str(i)]["sickness_name"] for i in range(len(docs_mapping))]

# Example usage
input_symptoms = "Runny nose, sneezing, sore throat"
retrieved_sickness = retriever(input_symptoms, index, sickness_names)

# Dictionary of the retrieved sickness
sicknesses[retrieved_sickness]

{'Description': 'A viral infection of the upper respiratory tract, primarily affecting the nose and throat.',
 'Symptoms': ['Runny or stuffy nose',
  'Sneezing',
  'Sore throat',
  'Coughing',
  'Mild headache',
  'Fatigue',
  'Low-grade fever'],
 'Causes': 'Caused by various viruses, most commonly rhinoviruses. Spreads through respiratory droplets or contact with contaminated surfaces.',
 'Medications': ['Decongestants (e.g., pseudoephedrine) for nasal congestion',
  'Pain relievers (e.g., ibuprofen or acetaminophen) for fever and headaches',
  'Cough suppressants (e.g., dextromethorphan)',
  'Rest, hydration, and soothing throat lozenges']}

# Token count ( good practice )

In [4]:
prompt = f"{sicknesses[retrieved_sickness]}"
model = genai.GenerativeModel("models/gemini-1.5-flash")

# Call `count_tokens` to get the input token count (`total_tokens`).
print("total_tokens: ", model.count_tokens(prompt))

total_tokens:  total_tokens: 150



# Exemple 

In [ ]:
def generate_response(query, retrieved_docs):
    model = genai.GenerativeModel('models/gemini-1.5-flash')    
    prompt = f"""Respond to the query as if you're having a friendly, informative conversation:
    Context:
    {' '.join(retrieved_docs)}
    Query: {query}
    Explain the answer in a warm, accessible manner that's easy to understand."""


    response = model.generate_content(prompt)
    return response.text


input_symptoms = "I sneeze all day ! what do i have ?"
retrieved_docs = retriever(input_symptoms, index, sickness_names)
response = generate_response(input_symptoms, f"{sicknesses[retrieved_sickness]}")

In [12]:
response

"Oh no, sneezing all day sounds really unpleasant!  It's impossible for me to tell you exactly what you have just from that symptom alone, since many things can cause excessive sneezing.  However, based on what you've described, it *sounds like* you might have a common cold.\n\nLooking at this information I have about the common cold, you'll see that one of the key symptoms is, indeed, a lot of sneezing.  Other symptoms often include a runny or stuffy nose, sore throat, cough, maybe a mild headache, and some fatigue.  Do any of those sound familiar?\n\nThe common cold is a viral infection of the upper respiratory tract – that's the nose and throat area. It's usually caused by rhinoviruses, and it spreads through tiny droplets in the air when someone coughs or sneezes, or by touching a contaminated surface.\n\nNow, it's important to remember I'm not a doctor, so I can't give medical advice.  If your sneezing is severe, persistent, or accompanied by other worrying symptoms like a high fe